In [1]:
import numpy as np
import tensorflow as tf
import os
import sys
from tensorflow.nn import rnn_cell
import pickle
import gzip
import random
import time
import shutil
import brain_python3 as bp
from datetime import datetime
import matplotlib.pyplot as plt

n_times = 201
n_frequency = 79
n_outputs = 2

last_neurons = 40
n_neurons = [40,40,last_neurons]
limit_part = 1500

high_max_load = 1

important_index = [105,125]
normal_range_index = [20,180]
window = 30
step = 1

learning_rate = 0.001
train_keep_prob = 1.0
want_round = 9
want_loop_percent = 0.9
test_keep_prob = 1.0
eeg = bp.brain_python(n_times,n_frequency,window,n_outputs,n_neurons, learning_rate)

data_base_path_array = ["./brain_data8~50_2048","./brain_data8~50_1024"]
train_file_path_array = ["./hen2048","./hen1024"]

for loop in range(len(train_file_path_array)):
    train_file_path=train_file_path_array[loop]
    data_base_path=data_base_path_array[loop]
    print(data_base_path)
    print(train_file_path)
    
    tf.set_random_seed(777)

    label_list,label_dic,label_onehot = eeg.label_info(data_base_path)
    _,_,test_label_onehot = eeg.label_info(data_base_path)

    number_of_round = 0
    remain_data = 0

    for label_list_name in label_list:
        remain_data += len(os.listdir(os.path.join(data_base_path,label_list_name)))

    if not os.path.isdir(train_file_path):
        os.makedirs(train_file_path)

    final_data_path = os.path.join(train_file_path,"final_data")

    if os.path.isdir(final_data_path):
        shutil.rmtree(final_data_path)

    os.makedirs(final_data_path)

    intersection_array = np.array([])

    test_data,test_data_label, number_label = eeg.preprocessing(data_base_path)
    train_data,train_data_label,_ = eeg.preprocessing(data_base_path)

    test_data = test_data[:,100:130]

    test_data=np.swapaxes(test_data,1,2)
    test_data=np.flip(test_data,axis=1)
    
    test_data_limit = np.zeros_like(test_data)
    
    for i in range(test_data.shape[0]):
        for j in np.arange(0,1,1/limit_part):
            test_data_limit[i]+=(test_data[i]>=(test_data[i].max()*j))
        
    while number_of_round<=want_round:

        random_pick = random.sample(range(0,train_data.shape[0]),int(train_data.shape[0]*0.5))
        final_data = train_data[random_pick]
        final_data_label = train_data_label[random_pick]

        final_data, final_data_label = eeg.data_multiple(final_data,final_data_label,label_list,important_index,normal_range_index,step)
        
        final_data=np.swapaxes(final_data,1,2)
        final_data=np.flip(final_data,axis=1)
        final_data_reverse = np.flip(final_data,axis=2)
        
        final_data=np.concatenate((final_data,final_data_reverse),axis=0)
        final_data_label=np.concatenate((final_data_label,final_data_label),axis=0)
        final_data_limit = np.zeros_like(final_data)
        
        for i in range(final_data.shape[0]):
            for j in np.arange(0,1,1/limit_part):
                final_data_limit[i]+=(final_data[i]>=(final_data[i].max()*j))

        batch_size = int(final_data.shape[0]*0.05)

        epoch = 0
        last_accuracy = 0.0
        convergence_array = np.zeros(standard_convergence)  
        saver_high = tf.train.Saver(max_to_keep=high_max_load)
        dif_array=np.zeros(batch_size)
        acc_array=np.zeros(batch_size)
        high_acc_array=np.zeros(batch_size)
        last_dif = 0
        end_percent = 0

        train_round_path = os.path.join(train_file_path,str(number_of_round)+"round")
        if not os.path.isdir(train_round_path):
            os.makedirs(train_round_path)

        high_ckpt_path=os.path.join(train_round_path,"high_ckpt")
        if not os.path.isdir(high_ckpt_path):
            os.makedirs(high_ckpt_path)

        train_x = np.zeros((batch_size,final_data.shape[1],final_data.shape[2]))
        train_y = np.zeros((batch_size,final_data_label.shape[1]))
        
        print(number_of_round)

        with tf.Session() as sess:
            tf.global_variables_initializer().run()

            if tf.train.get_checkpoint_state(high_ckpt_path) and tf.train.get_checkpoint_state(high_ckpt_path).model_checkpoint_path:
                saver_high.restore(sess, os.path.join(high_ckpt_path,str(number_of_round)))
                accuracy,_,_,_,y_pred = eeg.accuracy(sess,test_data_limit,test_data_label, test_keep_prob,label_onehot,test_label_onehot)
                print("accuracy"+str(accuracy))

            else:
                while end_percent<0.98:
                    start = time.time()
                    avr_loss = 0
                    for n in range(final_data.shape[0]//batch_size):
                        for i in range(batch_size):
                            data_num = random.randint(0,(final_data.shape[0]-1))
                            train_x[i] = final_data_limit[data_num,:]
                            train_y[i] = final_data_label[data_num]

                        loss = eeg.train(sess,train_x,train_y,train_keep_prob)

                        avr_loss += loss
                        epoch_percentage = int((n+1)*100/(final_data.shape[0]//batch_size))
                        print("["+("|"*epoch_percentage)+(" "*(100-epoch_percentage))+"]\tSCE: {:6f}".format(loss),end="\r")

                    avr_loss = avr_loss/(final_data.shape[0]//batch_size)
                    print('\nstep: {:03d}, SCE: {:6f}'.format(epoch, avr_loss))

                    accuracy,_,_,_,_ = eeg.accuracy(sess,test_data_limit,test_data_label, test_keep_prob,label_onehot,test_label_onehot)

                    if epoch == 0:
                        top_sce = avr_loss
                        bottom_sce = avr_loss

                    if top_sce < avr_loss:
                        top_sce = avr_loss
                    if bottom_sce > avr_loss:
                        bottom_sce = avr_loss

                    if (accuracy >= last_accuracy) and (avr_loss<((top_sce+bottom_sce)/2)) :
                        last_accuracy = accuracy
                        saver_high.save(sess, os.path.join(high_ckpt_path,str(number_of_round)))
                        print("Save high Model")

#                     if (epoch%25 is 24) and (avr_loss<((top_sce+bottom_sce)/2)):
#                         saver_high.save(sess, os.path.join(high_ckpt_path,str(number_of_round)))

                    ref_dif = last_accuracy/10
                    acc_array[epoch%batch_size] = accuracy
                    high_acc_array[epoch%batch_size] = last_accuracy

                    dif = (sum(acc_array)/batch_size)*(sum(high_acc_array)/batch_size+1)
                    dif_array[epoch%batch_size]=dif-last_dif
                    last_dif = dif

                    convergence_array[epoch%standard_convergence] = ref_dif>sum(dif_array)
                    end_percent = sum(convergence_array)/standard_convergence
                    epoch += 1
                    end = time.time()
                    print('Round : {},accuracy: {:6f}, epoch time: {:6f}, end_percnet: {:6f}'.format(number_of_round,accuracy,end-start,end_percent))

            saver_high.restore(sess, os.path.join(high_ckpt_path,str(number_of_round)))
            _,correct_index,_,_,_ = eeg.accuracy(sess,test_data_limit,test_data_label, test_keep_prob,label_onehot,test_label_onehot)
            success_data_path = os.path.join(train_round_path,"success_data")

            if os.path.isdir(success_data_path):
                shutil.rmtree(success_data_path)

            os.makedirs(success_data_path)

            for label_name in label_list:
                os.makedirs(os.path.join(success_data_path,label_name))

            for i in correct_index:
                start_path=os.path.join(data_base_path,label_onehot[np.argmax(test_data_label[i])],number_label[i])
                arrive_path=os.path.join(success_data_path,label_onehot[np.argmax(test_data_label[i])])
                shutil.copy(start_path,arrive_path)

            intersection_array = np.append(intersection_array,success_data_path)

            number_of_round += 1

    for label_list_name in label_list:
        final_path=os.path.join(final_data_path,label_list_name)
        if os.path.isdir(final_path):
            shutil.rmtree(final_path)
        os.makedirs(final_path)
        intersection_data = np.array([])
        for path in intersection_array:
            intersection_data=np.append(intersection_data,os.listdir(os.path.join(path,label_list_name)))
        unique, counts = np.unique(intersection_data, return_counts=True)
        for i,j in zip(unique, counts):
            if j>=int((want_round+1)*want_loop_percent):
                shutil.copy(os.path.join(data_base_path,label_list_name,i),final_path)

    #마지막 학습

    train_data,train_data_label,_ = eeg.preprocessing(final_data_path)
    test_data,test_data_label, number_label = eeg.preprocessing(data_base_path)

    final_data, final_data_label = eeg.data_multiple(train_data,train_data_label,label_list,important_index,normal_range_index,step)
    test_data = test_data[:,100:130]
    
    final_data=np.swapaxes(final_data,1,2)
    final_data=np.flip(final_data,axis=1)
    final_data_reverse = np.flip(final_data,axis=2)
    final_data=np.concatenate((final_data,final_data_reverse),axis=0)
    final_data_label=np.concatenate((final_data_label,final_data_label),axis=0)
    final_data_limit = np.zeros_like(final_data)
    
    
    test_data=np.swapaxes(test_data,1,2)
    test_data=np.flip(test_data,axis=1)
    test_data_limit = np.zeros_like(test_data)
    
    for i in range(final_data.shape[0]):
        for j in np.arange(0,1,1/limit_part):
            final_data_limit[i]=(final_data[i]>=(final_data[i].max()*j))

    for i in range(test_data.shape[0]):
        for j in np.arange(0,1,1/limit_part):
            test_data_limit[i]=(test_data[i]>=(test_data[i].max()*j))

    batch_size = int(final_data.shape[0]*0.05)

    epoch = 0
    last_accuracy = 0.0
    convergence_array = np.zeros(standard_convergence)  
    saver_high = tf.train.Saver(max_to_keep=high_max_load)
    dif_array=np.zeros(batch_size)
    acc_array=np.zeros(batch_size)
    high_acc_array=np.zeros(batch_size)
    last_dif = 0
    end_percent = 0

    train_round_path = os.path.join(train_file_path,"final round")
    if not os.path.isdir(train_round_path):
        os.makedirs(train_round_path)

    high_ckpt_path=os.path.join(train_round_path,"high_ckpt")
    if not os.path.isdir(high_ckpt_path):
        os.makedirs(high_ckpt_path)

    train_x = np.zeros((batch_size,final_data.shape[1],final_data.shape[2]))
    train_y = np.zeros((batch_size,final_data_label.shape[1]))

    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        if tf.train.get_checkpoint_state(high_ckpt_path) and tf.train.get_checkpoint_state(high_ckpt_path).model_checkpoint_path:
            pass

        else:  
            while end_percent<0.98:
                start = time.time()
                avr_loss = 0
                for n in range(final_data.shape[0]//batch_size):
                    for i in range(batch_size):
                        data_num = random.randint(0,(final_data.shape[0]-1))
                        train_x[i] = final_data_limit[data_num,:]
                        train_y[i] = final_data_label[data_num]

                    loss = eeg.train(sess,train_x,train_y,train_keep_prob)

                    avr_loss += loss
                    epoch_percentage = int((n+1)*100/(final_data.shape[0]//batch_size))
                    print("["+("|"*epoch_percentage)+(" "*(100-epoch_percentage))+"]\tSCE: {:6f}".format(loss),end="\r")

                avr_loss = avr_loss/(final_data.shape[0]//batch_size)
                print('\nstep: {:03d}, SCE: {:6f}'.format(epoch, avr_loss))

                accuracy,_,_,_,y_pred = eeg.accuracy(sess,test_data_limit,test_data_label, test_keep_prob,label_onehot,test_label_onehot)

                if epoch == 0:
                    top_sce = avr_loss
                    bottom_sce = avr_loss

                if top_sce < avr_loss:
                    top_sce = avr_loss
                if bottom_sce > avr_loss:
                    bottom_sce = avr_loss

                if (accuracy >= last_accuracy) and (avr_loss<((top_sce+bottom_sce)/2)) :
                    last_accuracy = accuracy
                    saver_high.save(sess, os.path.join(high_ckpt_path,str(number_of_round)))
                    print("Save high Model")

                ref_dif = last_accuracy/10
                acc_array[epoch%batch_size] = accuracy
                high_acc_array[epoch%batch_size] = last_accuracy

                dif = (sum(acc_array)/batch_size)*(sum(high_acc_array)/batch_size+1)
                dif_array[epoch%batch_size]=dif-last_dif
                last_dif = dif

                convergence_array[epoch%standard_convergence] = ref_dif>sum(dif_array)
                end_percent = sum(convergence_array)/standard_convergence
                epoch += 1
                end = time.time()
                print('Round : {},accuracy: {:6f}, epoch time: {:6f}, end_percnet: {:6f}'.format(number_of_round,accuracy,end-start,end_percent))

        saver_high.restore(sess, os.path.join(high_ckpt_path,str(number_of_round)))
        _,correct_index,fail_index,unknown_index,_ = eeg.accuracy(sess,test_data_limit,test_data_label, test_keep_prob,label_onehot,test_label_onehot)
        success_data_path = os.path.join(train_round_path,"success_data")
        fail_data_path = os.path.join(train_round_path,"fail_data")
        unknown_data_path = os.path.join(train_round_path,"unknown_data")

        final_round_path=[success_data_path,fail_data_path,unknown_data_path]
        final_round_index=[correct_index,fail_index,unknown_index]

        for i,j in zip(final_round_path,final_round_index):
            if os.path.isdir(i):
                shutil.rmtree(i)
            os.makedirs(i)

            for label_name in label_list:
                os.makedirs(os.path.join(i,label_name))

            for index in j:
                start_path=os.path.join(data_base_path,label_onehot[np.argmax(test_data_label[index])],number_label[index])
                arrive_path=os.path.join(i,label_onehot[np.argmax(test_data_label[index])])
                shutil.copy(start_path,arrive_path)


C:\Users\ADM\.conda\envs\yng6012_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ADM\.conda\envs\yng6012_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ADM\.conda\envs\yng6012_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ADM\.conda\envs\yng6012_gpu\lib\site-packages\tensorflow\pyth

./hen2048
./brain_data8~50
action
normal
action
normal
0
INFO:tensorflow:Restoring parameters from ./hen2048\0round\high_ckpt\0
accuracy0.5199004975124378
INFO:tensorflow:Restoring parameters from ./hen2048\0round\high_ckpt\0


KeyboardInterrupt: 

In [ ]:
exit()